In [1]:
import pandas as pd
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
train.Building_Painted=train.Building_Painted.map(dict(V=1, N=0))
train.Building_Fenced=train.Building_Fenced.map(dict(V=1, N=0))
train.Garden=train.Garden.map(dict(V=1, O=0))
train.Settlement=train.Settlement.map(dict(U=1, R=0))
train.NumberOfWindows=train.NumberOfWindows.replace({'   .': 0,'>=10':13})

Geo_Code=[]

for index, row in train.iterrows():
    row['Geo_Code']=str(row['Geo_Code']).replace('A','10')
    row['Geo_Code']=str(row['Geo_Code']).replace('B','11')
    #train1.iloc[index]['Geo_Code']=row['Geo_Code']
    #print(row['Geo_Code'])
    Geo_Code.append(row['Geo_Code'])
df_Geo_Code=pd.DataFrame({'Geo_Code':Geo_Code})

train['Geo_Code']=Geo_Code
train=train.dropna()


train_Y=train['Claim']
train_X=train.drop('Claim', 1)
train_X=train_X.drop('Customer Id', 1)

Customer=test['Customer Id']

test.Building_Painted=test.Building_Painted.map(dict(V=1, N=0))
test.Building_Fenced=test.Building_Fenced.map(dict(V=1, N=0))
test.Garden=test.Garden.map(dict(V=1, O=0))
test.Settlement=test.Settlement.map(dict(U=1, R=0))
test.NumberOfWindows=test.NumberOfWindows.replace({'   .': 0,'>=10':13})

Geo_Code=[]

for index, row in test.iterrows():
    row['Geo_Code']=str(row['Geo_Code']).replace('A','10')
    row['Geo_Code']=str(row['Geo_Code']).replace('B','11')
    #train1.iloc[index]['Geo_Code']=row['Geo_Code']
    #print(row['Geo_Code'])
    if not row['Geo_Code']:
      row['Geo_Code']=0
    Geo_Code.append(row['Geo_Code'])
df_Geo_Code=pd.DataFrame({'Geo_Code':Geo_Code})

test['Geo_Code']=Geo_Code
test=test.fillna(0)
test=test.interpolate(method ='linear', limit_direction ='forward')
test.to_csv('ctt.csv',index=False)
test=test.dropna()
val_Sub=test['Customer Id']
test.to_csv('tt.csv',index=False)
test1=test.drop('Customer Id', 1)



In [2]:
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_Y, test_size=0.1, random_state=44)


In [3]:
X_train.shape

(5888, 12)

In [4]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_T = sc.fit_transform(X_train)
X_Te = sc.fit_transform(X_test)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
Y_T = ohe.fit_transform(Y_train.values.reshape(-1, 1)).toarray()
Y_Te = ohe.fit_transform(Y_test.values.reshape(-1, 1)).toarray()

In [ ]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense# Neural network
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(30, activation='sigmoid'))
#model.add(Dropout(0.25))
#model.add(Dense(30, activation='softsign'))
#model.add(Dense(30, activation='sigmoid'))
#model.add(Dense(30, activation='tanh'))
#model.add(Dense(30, activation='selu'))
model.add(Dense(30, activation='sigmoid'))

model.add(Dense(2, activation='sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_T, Y_T, epochs=1000, batch_size=100)

In [ ]:
y_pred = model.predict(X_Te)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(Y_Te)):
    test.append(np.argmax(Y_Te[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

In [ ]:
pred

In [ ]:
val_x = sc.fit_transform(test1)
val_y=model.predict(val_x)

In [ ]:
val_y

In [ ]:
pred_ = list()
for i in range(len(val_y)):
    pred_.append(np.argmax(val_y[i]))

In [ ]:
pred_

In [ ]:
cal_claim=pd.DataFrame({'Customer Id':val_Sub,'Claim':pred_})

In [ ]:
cal_claim.to_csv('nnSub.csv',index=False)